In [0]:
rest_df = spark.read.format("json")\
    .option("multiline","true")\
    .option("inferschema","true")\
    .load("/FileStore/tables/resturant_json_data.json")



In [0]:
rest_df.show()

+----+-------+--------------------+-------------+-------------+-------------+------+
|code|message|         restaurants|results_found|results_shown|results_start|status|
+----+-------+--------------------+-------------+-------------+-------------+------+
|null|   null|                  []|            0|            0|            1|  null|
|null|   null|[{{{17066603}, b9...|         6835|           20|            1|  null|
|null|   null|                  []|            0|            0|            1|  null|
|null|   null|                  []|            0|            0|            1|  null|
|null|   null|[{{{17093124}, b9...|         8680|           20|            1|  null|
|null|   null|                  []|            0|            0|            1|  null|
|null|   null|                  []|            0|            0|            1|  null|
|null|   null|[{{{17580142}, b9...|          943|           20|            1|  null|
|null|   null|                  []|            0|            0|  

In [0]:
# Printing the schema to see which fields do we need to flatten in the json data, flattening means storing all values at the same level without
# nesting etc.
rest_df.printSchema()

root
 |-- code: long (nullable = true)
 |-- message: string (nullable = true)
 |-- restaurants: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- restaurant: struct (nullable = true)
 |    |    |    |-- R: struct (nullable = true)
 |    |    |    |    |-- res_id: long (nullable = true)
 |    |    |    |-- apikey: string (nullable = true)
 |    |    |    |-- average_cost_for_two: long (nullable = true)
 |    |    |    |-- cuisines: string (nullable = true)
 |    |    |    |-- currency: string (nullable = true)
 |    |    |    |-- deeplink: string (nullable = true)
 |    |    |    |-- establishment_types: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- events_url: string (nullable = true)
 |    |    |    |-- featured_image: string (nullable = true)
 |    |    |    |-- has_online_delivery: long (nullable = true)
 |    |    |    |-- has_table_booking: long (nullable = true)
 |    |    |    |-- i

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
# Arrays in json are flattened using the explode function,This will transform each element of an array into a separate row.
# Observe how after exploding it turned the data type into struct with which we can now easily access the data using .

rest_df.select("*",explode("restaurants").alias("new_restaurants")).printSchema()

root
 |-- code: long (nullable = true)
 |-- message: string (nullable = true)
 |-- restaurants: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- restaurant: struct (nullable = true)
 |    |    |    |-- R: struct (nullable = true)
 |    |    |    |    |-- res_id: long (nullable = true)
 |    |    |    |-- apikey: string (nullable = true)
 |    |    |    |-- average_cost_for_two: long (nullable = true)
 |    |    |    |-- cuisines: string (nullable = true)
 |    |    |    |-- currency: string (nullable = true)
 |    |    |    |-- deeplink: string (nullable = true)
 |    |    |    |-- establishment_types: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- events_url: string (nullable = true)
 |    |    |    |-- featured_image: string (nullable = true)
 |    |    |    |-- has_online_delivery: long (nullable = true)
 |    |    |    |-- has_table_booking: long (nullable = true)
 |    |    |    |-- i

In [0]:
rest_df.drop("restaurants")

Out[10]: DataFrame[code: bigint, message: string, results_found: bigint, results_shown: bigint, results_start: string, status: string]

In [0]:
#As you can see we accessed the specific column res_id using just . 
#It was possible cause we made it struct type
rest_df.select("*",explode("restaurants").alias("new_restaurants"))\
    .select("new_restaurants.restaurant.R.res_id").show()

+--------+
|  res_id|
+--------+
|17066603|
|17059541|
|17064405|
|17057797|
|17057591|
|17064266|
|17060516|
|17060320|
|17059060|
|17059012|
|17060869|
|17061231|
|17058534|
|17057925|
|17064031|
|17061237|
|17061253|
|17061296|
|17061205|
|17057397|
+--------+
only showing top 20 rows



In [0]:
#If you are not gettind any data on show that means explode has discarded all data cause explode discards data with null, in that case
#use explode_outer
rest_df.select("*",explode_outer("restaurants").alias("new_restaurants"))\
    .select("*","new_restaurants.restaurant.R.res_id").show()

+----+-------+--------------------+-------------+-------------+-------------+------+--------------------+--------+
|code|message|         restaurants|results_found|results_shown|results_start|status|     new_restaurants|  res_id|
+----+-------+--------------------+-------------+-------------+-------------+------+--------------------+--------+
|null|   null|[{{{17066603}, b9...|         6835|           20|            1|  null|{{{17066603}, b90...|17066603|
|null|   null|[{{{17066603}, b9...|         6835|           20|            1|  null|{{{17059541}, b90...|17059541|
|null|   null|[{{{17066603}, b9...|         6835|           20|            1|  null|{{{17064405}, b90...|17064405|
|null|   null|[{{{17066603}, b9...|         6835|           20|            1|  null|{{{17057797}, b90...|17057797|
|null|   null|[{{{17066603}, b9...|         6835|           20|            1|  null|{{{17057591}, b90...|17057591|
|null|   null|[{{{17066603}, b9...|         6835|           20|            1|  n